In [1]:
# We will need some functionality 
from typing import List 

# and from math related libraries
import numpy as np
import qutip as qt

# and from qiskit
from qiskit.extensions import HamiltonianGate
from qiskit import QuantumCircuit, QuantumRegister, Aer, execute
from qiskit.providers.aer import QasmSimulator
from qiskit.quantum_info import Operator

In [2]:
import numpy as np
import pylab

from qiskit import Aer
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.algorithms import VQE, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import SPSA
from qiskit.circuit.library import TwoLocal
from qiskit.opflow import I, X, Z

In [3]:
H2_op = (-1.052373245772859 * I ^ I) + \
        (0.39793742484318045 * I ^ Z) + \
        (-0.39793742484318045 * Z ^ I) + \
        (-0.01128010425623538 * Z ^ Z) + \
        (0.18093119978423156 * X ^ X)

print(f'Number of qubits: {H2_op.num_qubits}')
print(H2_op.to_matrix())

Number of qubits: 2
[[-1.06365335+0.j  0.        +0.j  0.        +0.j  0.1809312 +0.j]
 [ 0.        +0.j -1.83696799+0.j  0.1809312 +0.j  0.        +0.j]
 [ 0.        +0.j  0.1809312 +0.j -0.24521829+0.j  0.        +0.j]
 [ 0.1809312 +0.j  0.        +0.j  0.        +0.j -1.06365335+0.j]]


In [4]:
npme = NumPyMinimumEigensolver()
result = npme.compute_minimum_eigenvalue(operator=H2_op)
ref_value = result.eigenvalue.real
print(f'Reference value: {ref_value:.5f}')

Reference value: -1.85728


In [5]:
seed = 170
iterations = 125
algorithm_globals.random_seed = seed
backend = Aer.get_backend('aer_simulator')
qi = QuantumInstance(backend=backend, seed_simulator=seed, seed_transpiler=seed) 

counts = []
values = []
def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)

ansatz = TwoLocal(rotation_blocks='ry', entanglement_blocks='cz')

In [6]:
from qiskit.circuit import Parameter, QuantumCircuit, QuantumRegister 

# First we need to define at least one variational parameter
theta = Parameter('a')
psi = Parameter('b')

no_qubits = H2_op.num_qubits

# Create a quantum circuit that is used for the ansatz
ansatz = QuantumCircuit(H2_op.num_qubits)

# add some single-qubit gates
for i in range(no_qubits-1):
    if i % 2 == 0:
        ansatz.rx(psi, i)

ansatz.rz(theta, range(no_qubits))
    
# add a chain of CNOTs (multi-qubit gates)
for i in range(no_qubits-1):
    ansatz.cx(i, i+1)
ansatz.cx(no_qubits-1,0)
#ansatz.barrier()


# If you have prepared a state before, you can combine it with your ansatz
# ansatz.compose(prepared_state, front=True, inplace=True)

print(ansatz)



     ┌───────┐┌───────┐     ┌───┐
q_0: ┤ Rx(b) ├┤ Rz(a) ├──■──┤ X ├
     ├───────┤└───────┘┌─┴─┐└─┬─┘
q_1: ┤ Rz(a) ├─────────┤ X ├──■──
     └───────┘         └───┘     


In [7]:
def create_zz_hamiltonian(num_qubits: int, connectivity: List[List[int]],
                              h_coeffs: List[float]) -> np.ndarray:
    """Creates a global Hamiltonian consisting of a sum of one (g * Z^i.Z^j) term per device connection.

    Args:
        num_qubits (int): number of qubits of the Hamiltonian
        connectivity (List[List[int]]): connectivity of the device
        h_coeffs (List[float]): list of coefficients of each ZZ term

    Returns:
        np.ndarray: tensored Hamiltonian
    """
    
    dim = 2 ** num_qubits
    num_connections = len(connectivity)
    zz_hamiltonian = np.zeros([dim, dim], dtype=np.complex128)

    # create a matrix that is the sum of ZZ operators on all connected qubits,
    # tensored with the identity
    for c in range(num_connections):
        ops_to_tensor = [qt.identity(2)] * num_qubits
        ops_to_tensor[connectivity[c][0]] = qt.sigmaz()
        ops_to_tensor[connectivity[c][1]] = qt.sigmaz()
        zz_hamiltonian += h_coeffs[c] * np.array(qt.tensor(ops_to_tensor))

    return zz_hamiltonian

In [8]:
num_qubits = 2
hamiltonian = create_zz_hamiltonian(num_qubits, [[0, 1]], [1.,1.]) 
analog_block = HamiltonianGate(data=hamiltonian, time=2) 

print(analog_block)

Instruction(name='hamiltonian', num_qubits=2, num_clbits=0, params=[array([[ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j, -1.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j, -1.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  1.+0.j]]), 2])


In [9]:
ansatz = ansatz.to_gate()

In [10]:
qr = QuantumRegister(num_qubits)
circ = QuantumCircuit(qr)

circ.append(ansatz, [0,1])


circ.draw()

┌──────────────────┐
q0_0: ┤0                 ├
      │  circuit-89(a,b) │
q0_1: ┤1                 ├
      └──────────────────┘

In [11]:
spsa = SPSA(maxiter=iterations)
vqe = VQE(circ, optimizer=spsa, callback=store_intermediate_result, quantum_instance=qi)
result = vqe.compute_minimum_eigenvalue(operator=H2_op)
print(f'VQE on Aer qasm simulator (no noise): {result.eigenvalue.real:.5f}')
print(f'Delta from reference energy value is {(result.eigenvalue.real - ref_value):.5f}')

VQE on Aer qasm simulator (no noise): -1.06700
Delta from reference energy value is 0.79027


In [12]:
circ.unitary(analog_block, [qr[0], qr[1]], label='analog block')
#circ.measure_all()

#As always, we can visualize our circuit using the `QuantumCircuit.draw()` method.
circ.append(ansatz, [0,1])
circ.draw()

┌──────────────────┐┌───────────────┐┌──────────────────┐
q0_0: ┤0                 ├┤0              ├┤0                 ├
      │  circuit-89(a,b) ││  analog block ││  circuit-89(a,b) │
q0_1: ┤1                 ├┤1              ├┤1                 ├
      └──────────────────┘└───────────────┘└──────────────────┘

In [13]:
spsa = SPSA(maxiter=iterations)
vqe = VQE(circ, optimizer=spsa, callback=store_intermediate_result, quantum_instance=qi)
result = vqe.compute_minimum_eigenvalue(operator=H2_op)
print(f'VQE on Aer qasm simulator (no noise): {result.eigenvalue.real:.5f}')
print(f'Delta from reference energy value is {(result.eigenvalue.real - ref_value):.5f}')

VQE on Aer qasm simulator (no noise): -1.09838
Delta from reference energy value is 0.75889


In [14]:
circ.unitary(analog_block, [qr[0], qr[1]], label='analog block')
circ.draw()

┌──────────────────┐┌───────────────┐┌──────────────────┐»
q0_0: ┤0                 ├┤0              ├┤0                 ├»
      │  circuit-89(a,b) ││  analog block ││  circuit-89(a,b) │»
q0_1: ┤1                 ├┤1              ├┤1                 ├»
      └──────────────────┘└───────────────┘└──────────────────┘»
«      ┌───────────────┐
«q0_0: ┤0              ├
«      │  analog block │
«q0_1: ┤1              ├
«      └───────────────┘

In [15]:
spsa = SPSA(maxiter=iterations)
vqe = VQE(circ, optimizer=spsa, callback=store_intermediate_result, quantum_instance=qi)
result = vqe.compute_minimum_eigenvalue(operator=H2_op)
print(f'VQE on Aer qasm simulator (no noise): {result.eigenvalue.real:.5f}')
print(f'Delta from reference energy value is {(result.eigenvalue.real - ref_value):.5f}')

VQE on Aer qasm simulator (no noise): -1.85060
Delta from reference energy value is 0.00667
